# 最大熵模型

### 杜小勤，武汉纺织大学数学与计算机学院，2020年03月01日
https://github.com/duxiaoqin/Lecture-notes-for-Machine-Learning

改进的迭代尺度（Improved Iterative Scaling, IIS）算法

In [24]:
import math
import copy

class MaximumEntropy:
    def __init__(self, max_step = 1000, threshold = 0.005):
        self.max_step = max_step #最大迭代步数
        self.threshold = threshold #收敛阈值，小于该阈值，停止迭代
        
    #计算样本点x的Zw(x)值，它是条件分布Pw(y|x)的归一化因子
    #x:向量
    def zwx(self, x):
        zx = 0
        for y in self.labels: #每个标签类别
            zx_y = 0
            for xi in x: #每个特征分量
                if (xi, y) in self.x_y:
                    zx_y += self.x_y[(xi, y)][1] #sum wi*fi(xi,y)
            zx += math.exp(zx_y)
        return zx
    
    #计算样本点x的条件分布Pw(y|x)
    #x:向量
    #y:标量
    def pyx(self, x, y):
        zx = self.zwx(x)
        zx_y = 0
        for xi in x: #每个特征分量
            if (xi, y) in self.x_y:
                zx_y += self.x_y[(xi, y)][1] #sum wi*fi(xi,y)
        return math.exp(zx_y) / zx
    
    #计算样本点分量xi的模型期望EPF:特征函数fi(xi,y)关于条件分布Pw(y|xi)(即最大熵模型)的期望，EPF = sum P(xi)Pw(y|xi)fi(xi,y)
    #xi:样本点x的分量
    #y:标量
    #P(xi)Pw(y|xi)=P(xi,y)，而从本类P(xi)和Pw(y|xi)的计算公式可以看出，它们分别等同于P(x)和Pw(y|x)，
    #另外，从fit函数中的P(xi,y)计算公式又可以看出P(xi,y)<=>P(x,y)（此处x表示xi所在的样本点）。
    #因此，最大熵模型需要满足约束条件：P(x)Pw(y|x)=P(x,y)，从公式推导可以看出这一点。
    def epf(self, xi, y, X):
        ep = 0
        for x in X:
            if xi not in x:
                continue #fi(x, y)=0
            pyx = self.pyx(x, y) #计算单个样本点的P(y|x)
            ep += pyx/len(X) #计算sum P(x)P(y|x)fi(x,y)，其中P(x)=1/N，fi(x,y)=1
        return ep
        
    #优化最大熵模型的对偶函数或对数似然函数
    def fit(self, X, Y):
        N = len(X) #样例个数
        M = len(X[0]) #样例的特征分量个数，此处假定所有样例的特征分量个数都一样（常数），最大熵模型允许样例的维度不一样
        self.labels = set(Y) #类别（不重复）

        #统计所有(x,y)对的相关信息
        self.x_y = {}
        for i_sample in range(N):
            y = Y[i_sample]
            for xi in X[i_sample]:
                if (xi, y) in self.x_y:
                    self.x_y[(xi, y)][0] += 1
                else:
                    self.x_y[(xi, y)] = [1, 0, 0] #分别表示特征(x,y)的次数、拉格朗日乘数w、联合期望EPF_HAT
        #计算所有(xi,y)对的联合期望EPF_HAT:特征函数fi(xi,y)关于经验分布P(xi,y)的期望，EPF_HAT = sum P(xi,y)fi(xi,y)
        #从本类P(xi,y)的计算公式可以看出，P(xi,y)<=>P(x,y)，此处x表示xi所在的样本点
        for x_y in self.x_y: #遍历每个特征对(xi,y)
            self.x_y[x_y][2] = self.x_y[x_y][0] / N
        
        #应用IIS算法优化对偶函数或对数似然函数L(P,w)
        for step in range(self.max_step):
            last_x_y = copy.deepcopy(self.x_y)
            #计算所有(x,y)对的模型期望EPF:特征函数fi(x,y)关于条件分布Pw(y|x)(即最大熵模型)的期望，EPF = sum P(x)Pw(y|x)fi(x,y)
            for x_y in self.x_y: #遍历每个特征对(x,y)
                ep = self.epf(*x_y, X) # 计算第i个特征的模型期望
                self.x_y[x_y][1] += math.log(self.x_y[x_y][2]/ep)/M
            #判断任意权值之差是否小于阈值
            stop = True
            for x_y in self.x_y:
                if abs(last_x_y[x_y][1]-self.x_y[x_y][1]) >= self.threshold:
                    stop = False
                    break
            if stop:
                print('After {} step, maximum entropy model training completed!'.format(step))
                break
        print()
        print(self.x_y)
        
    def predict(self, X):
        results = []
        for x in X:
            zx = self.zwx(x)
            result = []
            for y in self.labels:
                zx_y = 0
                for xi in x:
                    if (xi, y) in self.x_y:
                        zx_y += self.x_y[(xi, y)][1]
                result.append((y, math.exp(zx_y)/zx))
            results.append(result)
        return results

生成数据集

In [25]:
import pandas as pd

dataset = [['sunny', 'hot', 'high', 'false', 'no'],
           ['sunny', 'hot', 'high', 'true', 'no'],
           ['overcast', 'hot', 'high', 'false', 'yes'],
           ['rainy', 'mild', 'high', 'false', 'yes'],
           ['rainy', 'cool', 'normal', 'false', 'yes'],
           ['rainy', 'cool', 'normal', 'true', 'no'],
           ['overcast', 'cool', 'normal', 'true', 'yes'],
           ['sunny', 'mild', 'high', 'false', 'no'],
           ['sunny', 'cool', 'normal', 'false', 'yes'],
           ['rainy', 'mild', 'normal', 'false', 'yes'],
           ['sunny', 'mild', 'normal', 'true', 'yes'],
           ['overcast', 'mild', 'high', 'true', 'yes'],
           ['overcast', 'hot', 'normal', 'false', 'yes'],
           ['rainy', 'mild', 'high', 'true', 'no']]
labels = ['Outlook', 'Temperature', 'Humidity', 'Windy', 'Play']
train_data = pd.DataFrame(dataset, columns = labels)
X = np.array(train_data.iloc[:, :-1])
Y = np.array(train_data.iloc[:, -1])
train_data

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,false,no
1,sunny,hot,high,true,no
2,overcast,hot,high,false,yes
3,rainy,mild,high,false,yes
4,rainy,cool,normal,false,yes
5,rainy,cool,normal,true,no
6,overcast,cool,normal,true,yes
7,sunny,mild,high,false,no
8,sunny,cool,normal,false,yes
9,rainy,mild,normal,false,yes


In [26]:
me = MaximumEntropy()
me.fit(X, Y)
me.predict([['overcast', 'mild', 'high', 'false'],
            ['overcast', 'hot', 'high', 'true'],
            ['overcast', 'hot', 'high', 'false'],
            ['rainy', 'hot', 'high', 'false'],
           ])

After 661 step, maximum entropy model training completed!

{('hot', 'no'): [2, 0.06209367542798627, 0.14285714285714285], ('high', 'yes'): [3, -2.238436267220732, 0.21428571428571427], ('hot', 'yes'): [2, 0.00647972101280744, 0.14285714285714285], ('rainy', 'no'): [2, 2.91587923674726, 0.14285714285714285], ('sunny', 'yes'): [2, -5.395840212106481, 0.14285714285714285], ('true', 'yes'): [3, -1.7212783901580846, 0.21428571428571427], ('mild', 'yes'): [4, 2.0403040677696396, 0.2857142857142857], ('false', 'yes'): [6, 1.634484168985822, 0.42857142857142855], ('cool', 'no'): [1, 3.6076010156607587, 0.07142857142857142], ('high', 'no'): [4, 1.5948247077075397, 0.2857142857142857], ('normal', 'yes'): [6, 1.8218352470476797, 0.42857142857142855], ('normal', 'no'): [1, -9.463290583750394, 0.07142857142857142], ('cool', 'yes'): [3, -1.4128887226475888, 0.21428571428571427], ('mild', 'no'): [2, -3.5142711774032054, 0.14285714285714285], ('overcast', 'yes'): [4, 5.262612273515946, 0.2857142857142

[[('yes', 0.9999971161857726), ('no', 2.8838142274653416e-06)],
 [('yes', 0.07986677792510691), ('no', 0.9201332220748931)],
 [('yes', 0.9992127716589034), ('no', 0.0007872283410965032)],
 [('yes', 0.047297708269788956), ('no', 0.952702291730211)]]

测试另一个数据集

In [27]:
datasets = [['youth-age', 'false', 'no', 'normal', 'reject'],
            ['youth-age', 'false', 'no', 'good', 'reject'],
            ['youth-age', 'true', 'no', 'good', 'accept'],
            ['youth-age', 'true', 'yes', 'normal', 'accept'],
            ['youth-age', 'false', 'no', 'normal', 'reject'],
            ['middle-age', 'false', 'no', 'normal', 'reject'],
            ['middle-age', 'false', 'no', 'good', 'reject'],
            ['middle-age', 'true', 'yes', 'good', 'accept'],
            ['middle-age', 'false', 'yes', 'excellent', 'accept'],
            ['middle-age', 'false', 'yes', 'excellent', 'accept'],
            ['old-age', 'false', 'yes', 'excellent', 'accept'],
            ['old-age', 'false', 'yes', 'good', 'accept'],
            ['old-age', 'true', 'no', 'good', 'accept'],
            ['old-age', 'true', 'no', 'excellent', 'accept'],
            ['old-age', 'false', 'no', 'normal', 'reject'],
           ]
labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
train_data = pd.DataFrame(datasets, columns = labels)
X = np.array(train_data.iloc[:, :-1])
Y = np.array(train_data.iloc[:, -1])
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,youth-age,false,no,normal,reject
1,youth-age,false,no,good,reject
2,youth-age,true,no,good,accept
3,youth-age,true,yes,normal,accept
4,youth-age,false,no,normal,reject
5,middle-age,false,no,normal,reject
6,middle-age,false,no,good,reject
7,middle-age,true,yes,good,accept
8,middle-age,false,yes,excellent,accept
9,middle-age,false,yes,excellent,accept


In [28]:
me = MaximumEntropy(max_step = 20000, threshold = 0.00005)
me.fit(X, Y)

After 19303 step, maximum entropy model training completed!

{('no', 'accept'): [3, -4.135416815833319, 0.2], ('middle-age', 'accept'): [3, 1.7527725553214912, 0.2], ('old-age', 'accept'): [4, 1.9334272600663023, 0.26666666666666666], ('excellent', 'accept'): [4, 5.495961967387707, 0.26666666666666666], ('normal', 'reject'): [4, 1.0993440265131085, 0.26666666666666666], ('false', 'reject'): [6, 2.807374818150756, 0.4], ('good', 'reject'): [2, -0.6898320995895466, 0.13333333333333333], ('false', 'accept'): [4, -4.060431387625057, 0.26666666666666666], ('good', 'accept'): [4, 0.38752990650040114, 0.26666666666666666], ('youth-age', 'accept'): [2, 3.0603392441365322, 0.13333333333333333], ('true', 'accept'): [5, 7.179910762509736, 0.3333333333333333], ('no', 'reject'): [6, 2.134461132899133, 0.4], ('yes', 'accept'): [6, 5.291723929592758, 0.4], ('old-age', 'reject'): [1, -6.853247773072396, 0.06666666666666667], ('middle-age', 'reject'): [2, -2.5009272993075147, 0.13333333333333333], ('yo

In [29]:
XTEST = [['old-age', 'false', 'no', 'normal'],
         ['middle-age', 'true', 'no', 'normal'],
         ['middle-age', 'false', 'no', 'normal'],
         ['youth-age', 'false', 'no', 'normal'],
         ['youth-age', 'false', 'no', 'good'],
         ['youth-age', 'false', 'yes', 'normal'],
         ['youth-age', 'false', 'yes', 'good'],
         ['old-age', 'false', 'yes', 'good'],
         ['middle-age', 'true', 'no', 'good'],
         ['youth-age', 'false', 'yes', 'good'],
         ['youth-age', 'true', 'yes', 'normal'],
        ]
me.predict(XTEST)

[[('accept', 8.166085756384233e-05), ('reject', 0.9999183391424362)],
 [('accept', 0.5254499081744641), ('reject', 0.47455009182553587)],
 [('accept', 8.77814832373704e-07), ('reject', 0.9999991221851676)],
 [('accept', 1.7531029402765733e-06), ('reject', 0.9999982468970597)],
 [('accept', 0.0008122299670197994), ('reject', 0.9991877700329802)],
 [('accept', 0.15544477552731326), ('reject', 0.8445552244726867)],
 [('accept', 0.9884183877857486), ('reject', 0.01158161221425133)],
 [('accept', 0.9997485348784596), ('reject', 0.00025146512154035214)],
 [('accept', 0.9980560640137665), ('reject', 0.001943935986233591)],
 [('accept', 0.9884183877857486), ('reject', 0.01158161221425133)],
 [('accept', 0.9999956927102314), ('reject', 4.307289768680757e-06)]]

参考文献：

- https://github.com/wzyonggege/statistical-learning-method
- 李航。《统计学习方法》，清华大学出版社，2012年3月第1版；